In [23]:
#1. 모듈 호출
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd 
import random
import os  
from selenium.webdriver.common.keys import Keys

In [24]:
#2. 사용자 인터페이스 만들기 
print('='*100)
print('유튜브 댓글 크롤러(대댓 제외)')
print('='*100)
key_word = input('1.유튜브에서 검색할 주제 키워드를 입력하세요(예:김어준의 뉴스공장) : ')
cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇 건 인가요? : '))
d_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇 건 입니까? : '))
f_dir = input('4.크롤링 결과를 저장할 폴더명만 쓰세요 : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+key_word)
os.chdir(f_dir+s+'-'+key_word)

f_name = s+'-'+key_word+'.txt'
fc_name = s+'-'+key_word+'.csv'
fx_name = s+'-'+key_word+'.xlsx'

s_time = time.time()

유튜브 댓글 크롤러(대댓 제외)
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:김어준의 뉴스공장) : 다꾸
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇 건 인가요? : 30
3.각 동영상에서 추출할 댓글은 몇 건 입니까? : 100
4.크롤링 결과를 저장할 폴더명만 쓰세요 : 


In [25]:
#3. 웹드라이버 호출 & 유튜브 접속
path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver88'
options=webdriver.ChromeOptions()
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')

driver = webdriver.Chrome(path, options=options)
driver.get('https://www.youtube.com/')
time.sleep(random.randrange(2, 5))

In [26]:
#4. 유튜브 검색창에 키워드 넣고 검색실행
element1 = driver.find_element_by_name('search_query')
element1.send_keys(key_word)
element1.send_keys('\n')
time.sleep(3)

In [27]:
#5. 링크 리스트 만들기 
link1 = []
no22 = 2
no11 = 1
if cnt <= 19 : 
    html0 = driver.page_source
    soup0 = BeautifulSoup(html0, 'html.parser')
    list0 = soup0.find_all('ytd-video-renderer', class_='style-scope ytd-item-section-renderer')
    time.sleep(3)
elif cnt > 19 : 
    #무한 스크롤 걸기
    while no22 == 2 : 
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        html0 = driver.page_source
        soup0 = BeautifulSoup(html0, 'html.parser')
        list0 = soup0.find_all('ytd-video-renderer', class_='style-scope ytd-item-section-renderer')
        time.sleep(2)
        if len(list0) >= cnt : 
            break

for a in list0 : 
    href = a.find('a', id='video-title')['href']
    link1.append(href)
    no11 += 1
    if no11 > cnt : 
        break
        
driver.quit()


In [28]:
#6. 영상 하나하나 들어가서 댓글 수집하기 
f = open(f_name, 'a', encoding='utf-8')
name1 = []
date1 = []
content1 = []
url1 = []
gogam1 = []
bgogam1 = []

a = 0
no44 = 1
no55 = 1

for b in link1 : 
    path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver88'
    options=webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

    driver = webdriver.Chrome(path, options=options)
    driver.get('https://www.youtube.com'+b)
    time.sleep(random.randrange(2, 5))
    
    driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN) #댓글 20개 발견
    time.sleep(8)
    html1 = driver.page_source 
    soup1 = BeautifulSoup(html1, 'html.parser')
    
    


    if d_cnt <= 20 : 
        dd_list = soup1.find('div', id='contents').find_all('ytd-comment-renderer', class_='style-scope ytd-comment-thread-renderer')
    
    elif d_cnt > 20 : 
        while no22 == 2 : 
            for z in range(1, 6) : 
                driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
                time.sleep(5)
            html5 = driver.page_source
            soup5 = BeautifulSoup(html5, 'html.parser')
            dd_list = soup5.find('div', id='contents').find_all('ytd-comment-renderer', class_='style-scope ytd-comment-thread-renderer')
            time.sleep(2)
            if len(dd_list) >= d_cnt : 
                break
            elif a == len(dd_list) : 
                print('%s 개 댓글 조회를 요청하셨으나, %s 개 밖에 없어서 %s 개 만큼만 조회하겠습니다' %(d_cnt, len(dd_list), len(dd_list)))
                break
            a = len(dd_list)
            

    for c in dd_list : #하나의 댓글 안에서 
        print('%s 번째 영상의 %s 번째 댓글' %(no44,no55 ))
        f.write('%s 번째 영상의 %s 번째 댓글' %(no44, no55) +'\n')
            
        print('-'*100)
        f.write('-'*100+'\n')
            
        url = 'https://www.youtube.com'+b
        url1.append(url)
        print('1.URL 주소 :',url)
        f.write('1.URL 주소 :'+url+'\n')
            
        name = c.find('a', id='author-text').get_text().replace('\n', "").strip()
        name1.append(name)
        print('2.댓글 작성자명 :',name)
        f.write('2.댓글 작성자 명 :'+name+'\n')
            
        date = c.find('a', class_='yt-simple-endpoint style-scope yt-formatted-string').get_text().replace('\n', "").strip()
        date1.append(date)
        print('3.댓글 작성일자 :',date)
        f.write('3.댓글 작성일자 :'+date+'\n')
            
        content = c.find('yt-formatted-string', id='content-text').get_text().replace('\n', "").strip()
        content1.append(content)
        print('4.댓글 내용 :',content)
        f.write('4.댓글 내용 :'+content+'\n')
        
        gogam = c.find('span', id='vote-count-middle').get_text().replace('\n', "").strip()
        if gogam == "" : 
            gogam = " "
        gogam1.append(gogam)
        print('5.공감 횟수 :',gogam)
        f.write('5.공감 횟수 :'+gogam+'\n')
        
        bgogam = c.find('div', id='creator-heart').get_text().replace('\n', "").strip()
        if bgogam == "" : 
            bgogam = " "
        bgogam1.append(bgogam)
        print('6.크리에이터 하트 :',bgogam)
        f.write('6.크리에이터 하트 :'+bgogam+'\n')
        f.write('\n')
        print('\n')

        no55 += 1
        if no55 > d_cnt : 
            break

            
    no44 += 1
    no55 = 1
    driver.quit()
f.close()
e_time = time.time()
total_time = e_time - s_time 

100 개 댓글 조회를 요청하셨으나, 90 개 밖에 없어서 90 개 만큼만 조회하겠습니다
1 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=YtmMGLoYyRA
2.댓글 작성자명 : 서윤꼬밍이
3.댓글 작성일자 : 1 day ago
4.댓글 내용 : 네잎클로버..뭔가 내일 나에게 행운이 올거같은..!모두 행운을 드려요~🍀
5.공감 횟수 : 18
6.크리에이터 하트 :  


1 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=YtmMGLoYyRA
2.댓글 작성자명 : Ju.hye_diary
3.댓글 작성일자 : 1 day ago
4.댓글 내용 : 금요일에 힐링하며 보는 졔리님 영상이라니,, 너무 좋습니다,,에 좋아요,,너무너무 조아요,,,,좋아 미치게써요♡♡♡♡
5.공감 횟수 : 4
6.크리에이터 하트 :  


1 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=YtmMGLoYyRA
2.댓글 작성자명 : 장동석
3.댓글 작성일자 : 1 day ago
4.댓글 내용 : 아깝게 33등ㅜㅠ네잎클로버🍀지에리님에게 행운을~🍀역시!!다꾸에 여신 후보 1위♥🍀
5.공감 횟수 : 1
6.크리에이터 하트 :  


1 번째 영상의 4 번째 댓글
------------------

6.크리에이터 하트 :  


1 번째 영상의 74 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=YtmMGLoYyRA
2.댓글 작성자명 : Yunjeong Park
3.댓글 작성일자 : 1 day ago
4.댓글 내용 : 일기 쓸 때 사용하신 펜 정보좀 알 수 있을까요??!!
5.공감 횟수 : 0
6.크리에이터 하트 :  


1 번째 영상의 75 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=YtmMGLoYyRA
2.댓글 작성자명 : 이시은
3.댓글 작성일자 : 1 day ago
4.댓글 내용 : 지에리님❤ 영상 넘 잘 보고 있답니당 댓 다는 건 처음인데 ٩(ˊᗜˋ*)و 항상 일기 쓰면서 보고 있는데 보면 항상 행복해요❣ 영상 올려주셔서 감사하고 코로나 항상 조심하시구😷 좋은 하루 보내세용 +먼슬리 영상 보다가 얼마전에 펜 샀는데 지에리님이랑 같은 펜이라서 너무 놀라고 좋았답니당😊
5.공감 횟수 : 0
6.크리에이터 하트 :  


1 번째 영상의 76 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=YtmMGLoYyRA
2.댓글 작성자명 : 나는사람
3.댓글 작성일자 : 1 day ago
4.댓글 내용 : 지에리님~~!!물어볼게 있어요!혹시 지에리님은 컨셉스티커(한장의 스티커로 다꾸를끝낼수있는스티커)를 

3 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=pr4m2zkiN2Q
2.댓글 작성자명 : 울배기
3.댓글 작성일자 : 4 months ago
4.댓글 내용 : 꾸미는것도 꾸미는거지만 작은 글씨로 예쁘게 반듯하고 일정하게 글씨쓰시는게 진짜 대박이시다..
5.공감 횟수 : 211
6.크리에이터 하트 :  


3 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=pr4m2zkiN2Q
2.댓글 작성자명 : 하하호호
3.댓글 작성일자 : 7 months ago
4.댓글 내용 : 저렇게 모아놓으면 뿌듯하고 이쁘네요.... 너무 부럽습니다... 저렇게 이쁘게 꾸미실줄 아시니..
5.공감 횟수 : 361
6.크리에이터 하트 : ❤ by 지에리 JIEREE TUBE


3 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=pr4m2zkiN2Q
2.댓글 작성자명 : 햄치즈
3.댓글 작성일자 : 2 weeks ago
4.댓글 내용 : 와 저렇게 꾸미려면 스티커 값이....
5.공감 횟수 : 20
6.크리에이터 하트 :  


3 번째 영상의 4 번째 댓글
-----------------------------------------------------------------------



3 번째 영상의 96 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=pr4m2zkiN2Q
2.댓글 작성자명 : 지민
3.댓글 작성일자 : 8 months ago
4.댓글 내용 : 어딜봐서 흑역사죠?🤔 (크리스마스날이 빠져있다!!)
5.공감 횟수 : 5
6.크리에이터 하트 : ❤ by 지에리 JIEREE TUBE


3 번째 영상의 97 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=pr4m2zkiN2Q
2.댓글 작성자명 : 일본현실반응
3.댓글 작성일자 : 8 months ago
4.댓글 내용 : 진짜 엄청 본받을게 많은 다꾸인것같아요ㅠㅠ 어쩜 이리 아이디어가 좋으신지ㅠㅠ
5.공감 횟수 : 3
6.크리에이터 하트 : ❤ by 지에리 JIEREE TUBE


3 번째 영상의 98 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=pr4m2zkiN2Q
2.댓글 작성자명 : 뿌뿌
3.댓글 작성일자 : 6 months ago
4.댓글 내용 : 혹시 사진 프린트는 따로 사용하시는 프린터기 있으ㅛㅣㄴ가요?
5.공감 횟수 : 0
6.크리에이터 하트 :  


3 번째 영상의 99 번째 댓글
-------------------------------------------------------------------------

5 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=UPHhibjH1Gk
2.댓글 작성자명 : 벨리곰TV - Bellygom TV
3.댓글 작성일자 : 3 months ago
4.댓글 내용 : 이ㅣ쁘게 쟈 ㄹ해따ㅏㅏ🤍
5.공감 횟수 : 279
6.크리에이터 하트 : ❤ by 옐언니


5 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=UPHhibjH1Gk
2.댓글 작성자명 : 신사장
3.댓글 작성일자 : 3 months ago
4.댓글 내용 : 와... 못하는게 대체 뭐닝...
5.공감 횟수 : 176
6.크리에이터 하트 :  


5 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=UPHhibjH1Gk
2.댓글 작성자명 : 오정애
3.댓글 작성일자 : 3 months ago
4.댓글 내용 : 제품들은 다이소가 좋지만 꾸미신거는 아트박스 게 더 이뿌게 잘꾸미신것 같아용❤️
5.공감 횟수 : 95
6.크리에이터 하트 :  


5 번째 영상의 4 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https:

...

100 개 댓글 조회를 요청하셨으나, 19 개 밖에 없어서 19 개 만큼만 조회하겠습니다
24 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=LQ2tCfGWtA8
2.댓글 작성자명 : dakku bebe다꾸베베
3.댓글 작성일자 : 1 week ago
4.댓글 내용 : 오늘도 재밌게 시청했어요☺️👍특히 율밤곰! 너무귀여워요😍좋은꿈꾸세요💕🌃
5.공감 횟수 : 5
6.크리에이터 하트 : ❤ by 제리니


24 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=LQ2tCfGWtA8
2.댓글 작성자명 : 아인슈페너
3.댓글 작성일자 : 1 week ago
4.댓글 내용 : 역시👍🏿첫번째다꾸는 분위기있는 내스타일💕두번째는 너무 귀엽고 예뽀 내스타일🥰세번째도 내가좋아하는 스타일 🥳네번째는 심플한 내스타일 💖다 너무예쁘고 멋져 보는내내 입가에 미소가 눈호강😍
5.공감 횟수 : 6
6.크리에이터 하트 : ❤ by 제리니


24 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=LQ2tCfGWtA8
2.댓글 작성자명 : ᄉᄋ
3.댓글 작성일자 : 1 week ago
4.댓글 내용 : 본격 다꾸 자극 영상!갑자기 미친듯이 하고 싶어집니다 ^__^특히 첫번째 다꾸 뭔가 차분하면서 와

100 개 댓글 조회를 요청하셨으나, 11 개 밖에 없어서 11 개 만큼만 조회하겠습니다
26 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=0vJ8jEO12_w
2.댓글 작성자명 : 다꾸소녀미주
3.댓글 작성일자 : 4 days ago
4.댓글 내용 : 그때 시간이 쫌 늦어서 이제 보내용ㅎㅎ오늘영상두 잼써여!그리구 솔리야님책상위에 수건?같은거 정보알려주셔서 샀어엿!감사해여,재밌는영상두 감사하고,다 감사드려용ㅎㅎ💖💖💖💖
5.공감 횟수 : 1
6.크리에이터 하트 : ❤ by 솔리야


26 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=0vJ8jEO12_w
2.댓글 작성자명 : 김경희
3.댓글 작성일자 : 13 hours ago
4.댓글 내용 : 아 텐바이텐 진짜 너무느려요 저도 엄청 화딱지 나더라구요...
5.공감 횟수 : 0
6.크리에이터 하트 :  


26 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=0vJ8jEO12_w
2.댓글 작성자명 : 김박력
3.댓글 작성일자 : 4 days ago
4.댓글 내용 : 젤리베어랑 마이버디 스티커2.0 미니버전 카톡선물하기에서 24장에 12900원이랍니다아!!! 하울영상보다가 반가워서 캬캬
5.공감 횟수 : 2
6.

4.댓글 내용 : 언니덕분에 다꾸의 재미를 알았어요퇴근하고 집에오면 항상 무기력해서 누워서 지냈는데 요새는 다이어리꾸미는것도 너무 즐겁고 일기를 쓰면서 자기반성도 하게되어서 하루를 보람차게 보내는것 같아요앞으로도 이쁜 영상 기대할게요
5.공감 횟수 : 2
6.크리에이터 하트 : ❤ by 차차노트 ASMR


27 번째 영상의 96 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=U_uiQsd_k3Q
2.댓글 작성자명 : 맥메이
3.댓글 작성일자 : 3 weeks ago
4.댓글 내용 : 힐링된당
5.공감 횟수 : 1
6.크리에이터 하트 : ❤ by 차차노트 ASMR


27 번째 영상의 97 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=U_uiQsd_k3Q
2.댓글 작성자명 : 아롱
3.댓글 작성일자 : 3 weeks ago (edited)
4.댓글 내용 : 차차님 따라 다꾸라는 걸 처음시작하고 빈티지다꾸로 연말을 가득 채우고있어요ㅎㅎㅎ 너무 재밌고 좋아요😆😆다이모까지 따라 사려고 찾아봤지만 다이모는 한국에서는 이미 사업을 철수한듯해서 직구로나마 겨우 다이모를 구했네요🤣차차님은 다이모를 오랫동안 써오신거겠죠? 소장가치가 꽤 있는 제품인 것 같아요 ㅎㅎ!!
5.공감 횟수 : 2
6.크리에이터 하트 : ❤ by 차차노트 ASMR


27 번째 영상의 98 번째 댓글
--------------------------------------------------------------------------------------------

29 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=itCwfbvSsmU
2.댓글 작성자명 : 이정인
3.댓글 작성일자 : 1 year ago
4.댓글 내용 : 무엇보다 저많은 스티커들과 메모지들의 출처를 알고 있다는게 신기해요ㅋㅋㅋ
5.공감 횟수 : 1.1K
6.크리에이터 하트 : ❤ by 보쨘BOJJYAN


29 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=itCwfbvSsmU
2.댓글 작성자명 : 인간
3.댓글 작성일자 : 1 year ago (edited)
4.댓글 내용 : 보쨘님: 자 다음 거 보여드릴게요나: 우와.. 엄청 잘하셧다.. 역시 보쨘님..보쨘님: Aㅏ.. 이거 좀 망했네요.나: ..?????
5.공감 횟수 : 812
6.크리에이터 하트 :  


29 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=itCwfbvSsmU
2.댓글 작성자명 : 김린
3.댓글 작성일자 : 1 year ago
4.댓글 내용 : 보쨘님 대학생인줄 알았는데  벌써 결혼까지 하셨다니 ㄷㄷ
5.공감 횟수 : 660
6.크리에이터 하트 : ❤ by 보쨘BOJJYAN


29 번째 영상의 4 번째 댓글
--------------------------------------

30 번째 영상의 1 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=PpD3X1_txxA
2.댓글 작성자명 : 이공LEEGONG
3.댓글 작성일자 : 1 year ago
4.댓글 내용 : 사랑해요!
5.공감 횟수 : 194
6.크리에이터 하트 : ❤ by 텐바이텐


30 번째 영상의 2 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=PpD3X1_txxA
2.댓글 작성자명 : 히히
3.댓글 작성일자 : 1 year ago
4.댓글 내용 : 미미 앞머리 없는게 찐이다... 대박...
5.공감 횟수 : 86
6.크리에이터 하트 :  


30 번째 영상의 3 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watch?v=PpD3X1_txxA
2.댓글 작성자명 : 톰리들
3.댓글 작성일자 : 1 year ago
4.댓글 내용 : 밈 혼자 촬영했나?? 단체샷에서 밈 혼자 어색  ㅋㅋ
5.공감 횟수 : 113
6.크리에이터 하트 : ❤ by 텐바이텐


30 번째 영상의 4 번째 댓글
----------------------------------------------------------------------------------------------------
1.URL 주소 : https://www.youtube.com/watc

In [29]:
#7. csv, xlsx 파일로 저장하기 
you = pd.DataFrame()
you['URL 주소'] = url1
you['댓글작성자명'] = name1
you['댓글작성일자'] = date1
you['댓글내용'] = content1
you['공감횟수'] = gogam1
you['크리에이터 하트'] = bgogam1

you.to_csv(fc_name, encoding='utf-8-sig')
you.to_excel(fx_name)

print('='*100)
print('총 %s 개 영상의 댓글을 수집하였습니다' %(cnt))
print('총 소요된 시간은 %s 초 입니다' %round(total_time, 1))
print('='*100)


총 30 개 영상의 댓글을 수집하였습니다
총 소요된 시간은 2973.2 초 입니다


19